# SQL Murder Mystery

The sql_murder_mystery database
A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it took place in ​SQL City​. Start by retrieving the corresponding crime scene report from the police department’s database.

<img src="https://mystery.knightlab.com/174092-clue-illustration.png?raw=true" width=100%/>

All the clues to this mystery are buried in a huge database, and you need to use SQL to navigate through this vast network of information. Your first step to solving the mystery is to retrieve the corresponding crime scene report from the police department’s database. Below we'll explain from a high level the commands you need to know; whenever you are ready, you can start adapting the examples to create your own SQL commands in search of clues -- you can run any SQL in any of the code boxes, no matter what was in the box when you started.

Image
Below is an Entity Relationship Diagram (ERD) of the sql-murder-mystery database:

<img src="https://mystery.knightlab.com/schema.png?raw=true" width=100%/>

Image
As can be seen from the ERD, the sql_murder_mystery database consists of 9 tables containing information about movies, cast, genre, and so much more.

Let's get started!

## Loading the database

Before you begin, you need to prepare your SQL environment.  You can do this by loading the magic command `%load_ext sql`.

In [1]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook. 
# If you get an error here, make sure that mysql and pymysql are installed correctly. 

%load_ext sql

Next, go ahead and load your database. To do this, you will need to ensure you have downloaded the `sql-murder-mystery.db` sqlite file from Athena and have stored it in a known location.

In [2]:
# Establish a connection to the local database using the '%sql' magic command.
# Replace 'password' with our connection password and `db_name` with our database name. 
# If you get an error here, please make sure the database name or password is correct.

%sql sqlite:///../databases/sql-murder-mystery.db

'Connected: @../databases/sql-murder-mystery.db'

### View all tables in the database

In [3]:
# View all tables present in the database
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///../databases/sql-murder-mystery.db
Done.


name
crime_scene_report
drivers_license
facebook_event_checkin
interview
get_fit_now_member
get_fit_now_check_in
solution
income
person


## The investigation begins.

Detective Lynn slumped into her worn-out chair, the exhaustion of a long day etched across her face. She stared at the stack of paperwork on her desk, filled with reports on missing cats and petty burglaries. She longed for something more substantial, a case that would truly challenge her investigative skills.

As she reached for her coffee mug, the door to her office swung open, and her boss, Captain Anderson, strode in with a stern expression. Lynn straightened up, ready to face whatever new challenge he had for her.

**Captain Anderson began _(his gaze penetrating)_**:. _"Detective Lynn, I need an update on the murder case I assigned you this morning. It's a high-profile one, and we need to solve it quickly."_

Lynn's heart raced. This was the opportunity she had been waiting for – a chance to delve into the darker side of crime and prove herself capable of handling more than just missing pets and minor thefts.

**Lynn**:_"Sir, I've been gathering information on the victim. I have also considered potential motives, but nothing concrete yet."_ 
Lynn replied, her fingers dancing across the keyboard as she pulled up her notes. 

He leans against her desk, his eyes narrowing.

**Captain Anderson**: _"We need results, Lynn. The mayor is breathing down my neck on this one. I want you to leave no stone unturned. Find the killer and bring them to justice."_.
He strides out of her office.

**Lynn** _(determination igniting in her eyes)_: _"I know that the crime was a **​murder​** that occurred sometime on **​Jan.15, 2018​** and that it took place in **​SQL City​.**"_ she mused. _"I need to find the crime scene report."_ 

In [10]:
%%sql
/* retrieve the corresponding crime scene report*/

SELECT *
FROM crime_scene_report
WHERE date = 20180115
AND type = 'murder'
AND city LIKE "SQL%";

 * sqlite:///../databases/sql-murder-mystery.db
Done.


date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


She glances at the crime scene report in her hands, the details of the murder case glaring back at her. Two witnesses – a potential link to the truth she seeks. 
**Lynn**: _"Hmmm, I need more information about these witnesses. But first, let me go over their statements, they might have provided more details about what they saw."_

In [5]:
%%sql
/* Check interview table for witness statements*/

SELECT p.name, i.transcript
FROM person p
JOIN interview i
ON p.id = i.person_id
WHERE (p.name LIKE "Annabel %" 
AND p.address_street_name LIKE "%Franklin%")
OR p.name = (
    SELECT p.name
    FROM person p
    JOIN interview as i
    ON p.id = i.person_id
    WHERE p.address_street_name LIKE "Northwestern Dr%"
    ORDER BY p.address_number DESC
    LIMIT 1
);

 * sqlite:///../databases/sql-murder-mystery.db
Done.


name,transcript
Morty Schapiro,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
Annabel Miller,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


**Detective Lynn**: _"I was right! The witnesses provided a treasure trove of information. Both witnesses mention that the killer was connected to a gym. Morty had information about the membership number and car. I need to get to that gym."_

The next day, Detective Lynn, clad in a sleek black jacket, stands outside the bustling city gym. 
The vibrant energy of fitness enthusiasts surrounds her, but her focus remains on the task at hand. 
She approaches the front desk, where a receptionist greets her with a friendly smile. 
Flashing her badge, Lynn requests access to the gym's member information.
The receptionist's demeanor shifts. Realizing the gravity of the situation, she hurridely puts a call to the Manager.

The Gym Manager approaches Lyn.
**Gym Manager** : _"Good day Ma'am, unfortunately it is against our policy to release information about our members. You need to come back with a search warrant."_

**Detective Lynn**: _"Listen, I'm not here to play games. I'm on the trail of a murderer, and there's a chance they're a member of your gym. If I don't get the information I need, I'll be forced to temporarily shut down this place for obstructing justice. Imagine the fallout – how do you think your members would react when they find out their gym is connected to a murder investigation?"_

Recognizing the urgency, the Gym Manager, fearing the repercussions of closure, eventually relents.

**Gym Manager**:_"Fine. But this better lead to something substantial."_

Lynn meticulously sifts through the gym records to find a gym member with a gold membership status, membership number starting with "48Z" and drove a car with a plate number that included "H42W". 

In [12]:
%%sql
/* 
To find the potential killer, join and query the drivers_license, get_fit_now_member, get_fit_now_check_in and person tables
*/

SELECT m.name, m.id, m.membership_status
FROM get_fit_now_member m
JOIN get_fit_now_check_in ci
ON m.id = ci.membership_id
JOIN person p
ON p.id = m.person_id
JOIN drivers_license dl
ON dl.id = p.license_id
WHERE m.id LIKE "48Z%"
AND m.membership_status = "gold"
AND dl.plate_number LIKE "%H42W%";

 * sqlite:///../databases/sql-murder-mystery.db
Done.


name,id,membership_status
Jeremy Bowers,48Z55,gold


Her dodgedness paid off! Detective Lynn identified Jeremy Bowers as the potential suspect. Swiftly, she gets more information about him, assembles a team and moves to apprehend Jeremy at his home. 

In [7]:
%%sql

SELECT * 
FROM person
WHERE name = "Jeremy Bowers";

 * sqlite:///../databases/sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


Detective Lynn and her team stand outside the suspect's apartment, the faint glow of a television and the muffled sounds of a movie leaking through

Lynn *(whispers to her team)*: _"Be ready. We go in on my signal."_

She knocks sharply on the door and the door creaks open. The room is dimly lit, with the eerie blue glow of the television casting shadows on the worn-out furniture. Jeremy, oblivious to their presence, sits engrossed on the sofa with empty beer cans strewn across the floor. Lynn and her team enter the apartment quietly.

**Lynn**: "Hands where I can see them!"

Jeremy is startled, a beer slipping from his hand and clattering to the floor.

**Lynn**: _"Stand up slowly, and keep your hands in the air."_

Jeremy Bowers: _"What's going on? Who are you people? How dare you barge into my house? I know my rights!"_

**Lynn** *(flashes her badge)*: _"Detective Lynn, Homicide. You are under arrest for murder."_

Jeremy shifts nervously, eyes darting around the apartment, looking for a way out. As Lynn moves to handcuff the suspect, he resists, pushing her to the floor. A struggle ensues but the team quickly brings the situation under control. She reads Jeremy his rights and escorts him, hands cuffed behind his back to the waiting car.

Finally at the police station, Jeremy is interrogated, which leads to even more revelations .

In [13]:
%%sql

SELECT p.name, i.transcript
FROM person p
JOIN interview i
ON p.id = i.person_id
WHERE p.name = "Jeremy Bowers";

 * sqlite:///../databases/sql-murder-mystery.db
Done.


name,transcript
Jeremy Bowers,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries.

Jeremy was hired by a lady with red hair who owns a Tesla Model S. I need to query the events table.

In [24]:
%%sql

SELECT DISTINCT(p.name)
FROM facebook_event_checkin fb
JOIN person p
ON fb.person_id = p.id
JOIN drivers_license dl
ON p.license_id = dl.id
WHERE fb.date like '201712%'
AND dl.hair_color = 'red'
AND dl.car_make = 'Tesla'
AND dl.car_model = 'Model S';

 * sqlite:///../databases/sql-murder-mystery.db
Done.


name
Miranda Priestly


Miranda Priestly is the Villian behind the scenes. She hired Jeremy Bowers to commit the crime.

## THE END.